In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical

!pip install scikit-plot
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

In [3]:
X_split = np.array_split(X, 4)
y_split = np.array_split(y, 4)

trainX_b, testX_b, trainY_b, testY_b = train_test_split(X_split[1], y_split[1], test_size=0.2)

In [4]:
model_b = Sequential()
model_b.add(Dense(3, activation='relu', input_shape=(trainX_b.shape[1], 1)))
model_b.add(Dropout(0.1))
model_b.add(Flatten())
model_b.add(Dense(1, activation='sigmoid'))
model_b.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_b.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21, 3)             6         
                                                                 
 dropout (Dropout)           (None, 21, 3)             0         
                                                                 
 flatten (Flatten)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 64        
                                                                 
Total params: 70 (280.00 Byte)
Trainable params: 70 (280.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
history_b = model_b.fit(trainX_b, trainY_b, epochs=3)

Epoch 1/3
1586/1586 [==============================] - 11s 6ms/step - loss: 0.6823 - accuracy: 0.7908
Epoch 2/3
1586/1586 [==============================] - 7s 5ms/step - loss: 0.3636 - accuracy: 0.8587
Epoch 3/3
1586/1586 [==============================] - 5s 3ms/step - loss: 0.3515 - accuracy: 0.8587


In [6]:
score_b = model_b.evaluate(testX_b, testY_b, verbose=0)
print(score_b)

[0.3424181640148163, 0.8599810600280762]


In [7]:
pesiTmp_b = model_b.get_weights()

In [8]:
import requests as r
import json

In [9]:
pesiTmp2_b = [pesiTmp_b[0][0], pesiTmp_b[1]]
# Conversione dei pesi per renderli compatibili con la codifica JSON e del server
pesi_b = '{\"l1\" : [[' + np.array2string(pesiTmp_b[0][0][0]) + '],[' + np.array2string(pesiTmp_b[0][0][1]) + '],[' + np.array2string(pesiTmp_b[0][0][2]) + ']], \"a\" : \"' + np.array2string(pesiTmp_b[1]) + '\"}'
pesi_json_b = json.loads(pesi_b)
print(pesi_b)

{"l1" : [[-0.3360691],[0.70658827],[0.1104556]], "a" : "[ 0.         -0.00265088  0.4254989 ]"}


In [10]:
url = 'http://93.65.162.11:3001'
x = r.post(url + '/nodo_b', data = pesi_b, headers={"Content-Type": "application/json"})
print(x)

<Response [200]>


In [ ]:
import json
media = r.get(url + '/pesi')
dict = json.loads(media.text)

pesi_tmp = np.array(dict['l1'])
pesi_tmp_b = np.array(dict['a'])

pesi_nodi = pesi_tmp.reshape((1,3))
pesi_archi = pesi_tmp_b.reshape((1,3))
pesi_archi[pesi_archi == None] = 0

In [ ]:
pesi_tmp_b = model_b.get_weights()

pesi_tmp_b[0][0][0] = pesi_nodi[0][0]
pesi_tmp_b[0][0][1] = pesi_nodi[0][1]
pesi_tmp_b[0][0][2] = pesi_nodi[0][2]

pesi_tmp_b[1][0] = pesi_archi[0][0]
pesi_tmp_b[1][1] = pesi_archi[0][1]
pesi_tmp_b[1][2] = pesi_archi[0][2]

In [ ]:
model_b.set_weights(pesi_tmp_b)
history_b = model_b.fit(trainX_b, trainY_b, epochs=3)
score_b = model_b.evaluate(testX_b, testY_b, verbose=0)